## DMM Result

In [3]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer
DMM = pickle.load(open('../../model_lda/DMM_result.pkl','rb'))

data = pickle.load(open('preprocessed_review.pickle', 'rb'))
data_join = [' '.join(doc) for doc in data]
cv = CountVectorizer()
X = cv.fit_transform(data_join).toarray()

DMM_lam = [DMM.components_[k,:] for k in range(4)]
def print_top_words(lam, feature_names, n_top_words):
    for topic_id, topic in enumerate(lam):
        print('\nTopic Nr.%d:' % int(topic_id + 1))
        print(''.join([feature_names[i] + ' ' + str(round(topic[i], 2))
                       + ' | ' for i in topic.argsort()[:-n_top_words - 1:-1]]))
print_top_words(DMM_lam, list(cv.get_feature_names()), 10)


Topic Nr.1:
hotel 3595.78 | stay 2803.74 | room 2764.67 | staff 1623.16 | great 1596.15 | service 1074.03 | waikiki 1015.44 | location 961.98 | beach 859.0 | place 831.88 | 

Topic Nr.2:
hotel 4903.6 | room 4674.14 | beach 3114.4 | stay 3002.51 | waikiki 2391.2 | view 2289.3 | great 2059.07 | nice 1425.27 | pool 1413.63 | location 1318.82 | 

Topic Nr.3:
hotel 7295.34 | great 5695.8 | room 4682.51 | stay 4162.87 | location 3840.33 | beach 3514.46 | staff 3322.45 | good 2555.8 | waikiki 2407.07 | nice 2009.29 | 

Topic Nr.4:
room 2199.08 | stay 2100.28 | hotel 1505.68 | staff 1337.29 | great 1107.37 | view 1024.0 | beach 826.53 | time 807.45 | waikiki 806.7 | location 740.27 | 


## CLDA Result

In [4]:
CLDA = pickle.load(open('../../model_lda/CLDA_result.pkl','rb'))
CLDA_lam =  [CLDA.components_[k,:] for k in range(4)]
print_top_words(CLDA_lam, list(cv.get_feature_names()), 10)


Topic Nr.1:
get 3589.1 | price 2652.0 | go 2405.05 | time 2400.06 | day 2306.27 | resort 2213.1 | value 1917.0 | pay 1873.0 | fee 1791.0 | just 1709.18 | 

Topic Nr.2:
staff 8591.0 | pool 4981.0 | service 4875.0 | friendly 4705.0 | helpful 3495.0 | check 3270.0 | stay 3206.75 | desk 2724.0 | make 2513.1 | parking 2103.0 | 

Topic Nr.3:
great 10297.07 | beach 8314.1 | location 6861.1 | waikiki 6593.93 | restaurant 4208.0 | breakfast 3596.0 | walk 3320.1 | good 2879.16 | food 2667.0 | place 2579.82 | 

Topic Nr.4:
hotel 17017.05 | room 15320.0 | stay 9862.25 | view 5256.1 | nice 4207.09 | clean 3346.04 | close 3118.0 | front 3069.0 | ocean 2423.1 | bed 2215.1 | 
